# Stratospheric aerosol optical depth at 550 nm

For some exercises we use measurements of the stratospheric aerosol optical depth (Sato et al., 1999).


In [ ]:
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def strat_aod(hemisphere):

    fN = 'isaod_{}.dat'.format(hemisphere)

    aod = pd.read_csv(fN, delim_whitespace=True, skiprows=3, 
        names=('year', 'month', 'AOD'))

   
    aod['day'] = 1
    
    date = pd.to_datetime(aod[['year', 'month', 'day']])
    
    
    ds = xr.DataArray(aod['AOD'], dims=('time',), 
                      coords=dict(time=('time', date)))
    
    ds = ds.to_dataset(name=hemisphere)

    ds = ds.groupby('time.year').mean('time')
    
    ds.attrs['data'] = 'Stratospheric aerosol optical depth at 550 nm'
    ds.attrs['source_nh'] = 'https://climexp.knmi.nl/data/isaod_nh.dat'
    ds.attrs['source_sh'] = 'https://climexp.knmi.nl/data/isaod_sh.dat'

    ds = ds.sel(year=slice(1875, None))
    
    return ds

In [ ]:
aod = strat_aod('nh')
sh = strat_aod('sh')

aod = aod.assign(sh=sh.sh)

aod.to_netcdf('aod.nc', format='NETCDF4_CLASSIC')

In [ ]:
volcanic_eruptions = (1883, 1902, 1912, 1963, 1982, 1991)
volcanoes = ('Krakatoa', 'Santa Maria', 'Novarupta', 'Agung', r'El Chich\'{o}n', 'Pinatubo')

In [ ]:
from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'custom'
matplotlib.rcParams['mathtext.rm'] = 'Bitstream Vera Sans'
matplotlib.rcParams['mathtext.it'] = 'Bitstream Vera Sans:italic'
matplotlib.rcParams['mathtext.bf'] = 'Bitstream Vera Sans:bold'


params = {'text.latex.preamble' : [r'']}


plt.rcParams.update(params)

In [ ]:
nh = strat_aod('nh')
sh = strat_aod('sh')


nh.nh.plot.line(label='nh')
sh.sh.plot.line(label='sh')


ax = plt.gca()


dx = 0.5
fontsize = 8

ax.set_ylim(None, 0.2)

for y, v in zip(volcanic_eruptions, volcanoes):
    ax.text(y - dx, 0.1925, v, fontsize=fontsize, rotation=90, va='top', ha='right')


#ax.set_xlim(1898.5, 2030)

for y in volcanic_eruptions:
    ax.axvline(y, color='0.1', lw=1)
    
# hack to convert labels from number to text
ax.set_xticklabels(['{:0g}'.format(tl) for tl in ax.get_xticks()])
ax.set_yticklabels(['{:0g}'.format(tl) for tl in ax.get_yticks()])

plt.legend()

plt.savefig('aod.pdf')